In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving npr.csv to npr.csv
User uploaded file "npr.csv" with length 55511119 bytes


In [4]:
# Preprocessing on npr dataset 
# www.npr.org
import pandas as pd
npr = pd.read_csv('npr.csv')
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [5]:
len(npr)
print(npr.shape)


(11992, 1)


In [6]:
npr['Article'][1]

'  Donald Trump has used Twitter  —   his preferred means of communication  —   to weigh in on a swath of foreign policy issues over the past few weeks. His comments give a glimpse into how his incoming administration will deal with pressing foreign matters  —   but also highlight how reactionary comments on social media can immediately spur international concern and attention. And his staff has indicated that taking to Twitter to air his concerns or, often, grievances, won’t end once he enters the Oval Office. On Wednesday, Trump blasted the U. S.’s abstention from the U. N. Security Council vote on Israeli settlements earlier this month. The tweets came just hours before Secretary of State John Kerry gave a speech defending the decision and calling the continued building of settlements on Palestinian territory in the West Bank a threat to the   solution in the region. Trump’s support for Israel and Prime Minister Benjamin Netanyahu  —   who has had a fraught relationship with Preside

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
dtm = cv.fit_transform(npr['Article'])
dtm

<11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

In [52]:
#from sklearn.decomposition import LatentDirichletAllocation
#LDA = LatentDirichletAllocation(n_components=7,random_state=42)
from sklearn.decomposition import NMF
LDA = NMF(n_components=7,random_state=42)
LDA.fit(dtm)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=7, random_state=42, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

# Three Steps Away...
### Grab the vocab.
### Grab the topic
### and finally grab the highest prob. word per topic



In [53]:
len(cv.get_feature_names())

54777

In [54]:
type(cv.get_feature_names())

list

In [55]:
cv.get_feature_names()[5510]

'beret'

In [56]:
import random

In [57]:
for i in range(10):
    random_word_id = random.randint(0,54776)
    print(cv.get_feature_names()[random_word_id])

frontal
ginny
decathlete
face
goma
armpit
regional
biplane
pie
232


### Showing Top Words Per Topic

In [60]:
len(LDA.components_)

7

In [61]:
LDA.components_.shape

(7, 54777)

In [62]:
LDA.components_

array([[2.55233049e-02, 0.00000000e+00, 4.90968123e-04, ...,
        1.16841424e-03, 1.18169925e-03, 2.91895163e-03],
       [7.36991412e-03, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 9.52098632e-01, 0.00000000e+00, ...,
        1.73051772e-03, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 7.52276930e-01, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.68552180e-02, 8.03450574e-01, 0.00000000e+00, ...,
        0.00000000e+00, 2.74473435e-04, 0.00000000e+00],
       [0.00000000e+00, 1.60926042e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [63]:
len(LDA.components_[0])

54777

In [20]:
single_topic = LDA.components_[0]

In [64]:
# Returns the indices that would sort this array.
single_topic.argsort()

array([ 2475, 18302, 35285, ..., 22673, 42561, 42993])

In [22]:
#lets understand the argsort in simple example
import numpy as np
arr = np.array([10,200,1])
arr.argsort()


array([2, 0, 1])

In [65]:
# Word least representative of this topic
single_topic[18302]

0.14285714309286987

In [68]:
# Word most representative of this topic
single_topic[42997]

2.1425726865107904

In [67]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

array([33390, 36310, 21228, 10425, 31464,  8149, 36283, 22673, 42561,
       42993])

In [69]:
top_word_indices = single_topic.argsort()[-10:]

In [71]:
for index in top_word_indices:
    print(cv.get_feature_names()[index])

new
percent
government
company
million
care
people
health
said
says


These look like business articles perhaps... Let's confirm by using .transform() on our vectorized articles to attach a label number. But first, let's view all the 10 topics found.

In [72]:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['new', 'lot', 'make', 'things', 've', 'don', 'way', 'time', 'going', 'really', 'know', 'think', 'just', 'people', 'like']


THE TOP 15 WORDS FOR TOPIC #1
['presidential', 'republicans', 'comey', 'administration', 'obama', 'new', 'election', 'republican', 'white', 'donald', 'house', 'campaign', 'said', 'president', 'trump']


THE TOP 15 WORDS FOR TOPIC #2
['government', 'say', 'city', 'day', 'university', 'year', 'water', 'food', 'time', 'just', 'new', 'years', 'like', 'people', 'says']


THE TOP 15 WORDS FOR TOPIC #3
['according', 'reports', 'news', 'department', 'new', 'law', 'time', 'government', 'house', 'npr', 'court', 'told', 'president', 'police', 'said']


THE TOP 15 WORDS FOR TOPIC #4
['country', 'election', 'new', 'states', 'vote', 'party', 'obama', 'democratic', 'voters', 'percent', 'campaign', 'hillary', 'state', 'sanders', 'clinton']


THE TOP 15 WORDS FOR TOPIC #5
['new', 'percent', 'year', 'parents', 'private', 'kids', 'children', 'student',

### Attaching Discovered Topic Labels to Original Articles

In [30]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

In [73]:
dtm.shape

(11992, 54777)

In [74]:
len(npr)

11992

In [75]:
topic_results = LDA.transform(dtm)

In [33]:
topic_results.shape

(11992, 7)

In [76]:
topic_results[0]

array([0.05459945, 0.88996904, 0.00661222, 0.24766001, 0.18977488,
       0.01347329, 0.        ])

In [77]:
topic_results[0].round(2)

array([0.05, 0.89, 0.01, 0.25, 0.19, 0.01, 0.  ])

In [78]:
topic_results[0].argmax()

1

This means that our model thinks that the first article belongs to topic #1.

### Combining with Original Data

In [ ]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [80]:
topic_results.argmax(axis=1)

array([1, 1, 1, ..., 3, 4, 2])

In [81]:
npr['Topic'] = topic_results.argmax(axis=1)

In [82]:
npr.head(20)

,Article,Topic
0,"In the Washington of 2016, even when the polic...",1
1,Donald Trump has used Twitter — his prefe...,1
2,Donald Trump is unabashedly praising Russian...,1
3,"Updated at 2:50 p. m. ET, Russian President Vl...",3
4,"From photography, illustration and video, to d...",5
5,I did not want to join yoga class. I hated tho...,0
6,With a who has publicly supported the debunk...,6
7,"I was standing by the airport exit, debating w...",2
8,"If movies were trying to be more realistic, pe...",2
9,"Eighteen years ago, on New Year’s Eve, David F...",2


##NON NEGATIVE MATRIX FACTORIZATION ALGORITHMS

In [40]:
#https://scikit-learn.org/stable/modules/decomposition.html#nmf
# dimensionality reducation
import numpy as np
from sklearn.decomposition import NMF
X = np.array([[1, 1], [2, 1], [3, 1.2], [4, 1], [5, 0.8], [6, 1]])


In [41]:
model = NMF(n_components=2, init='random', random_state=0)


In [42]:
W = model.fit_transform(X)
H = model.components_


In [44]:
W

array([[0.        , 0.46880684],
       [0.55699523, 0.3894146 ],
       [1.00331638, 0.41925352],
       [1.6733999 , 0.22926926],
       [2.34349311, 0.03927954],
       [2.78981512, 0.06911798]])

In [45]:
H

array([[2.09783018, 0.30560234],
       [2.13443044, 2.13171694]])

In [46]:
X_new = np.array([[1, 0], [1, 6.1], [1, 0], [1, 4], [3.2, 1], [0, 4]])
W_new = model.transform(X_new)

In [47]:
X_new

array([[1. , 0. ],
       [1. , 6.1],
       [1. , 0. ],
       [1. , 4. ],
       [3.2, 1. ],
       [0. , 4. ]])

In [48]:
W_new

array([[0.46677737, 0.        ],
       [0.        , 1.66350401],
       [0.46677737, 0.        ],
       [0.        , 1.17156989],
       [1.2277735 , 0.29280011],
       [0.        , 0.93701736]])

Using scikit-learn. We use the built-in 20 newsgroups dataset and see what topics are generated.

In [49]:
#https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

In [50]:
data= fetch_20newsgroups(remove=('headers', 'footers', 'quotes')).data

In [51]:
# convert the text to a tf-idf weighted term-document matrix
 
vectorizer = TfidfVectorizer(max_features=2000, min_df=10, stop_words='english')
X = vectorizer.fit_transform(data)
idx_to_word = np.array(vectorizer.get_feature_names())
 
# apply NMF
 
nmf = NMF(n_components=20, solver="mu")
W = nmf.fit_transform(X)
H = nmf.components_
 
# print the topics
 
for i, topic in enumerate(H):
    print("Topic {}: {}".format(i + 1, ",".join([str(x) for x in idx_to_word[topic.argsort()[-10:]]])))

Topic 1: think,said,right,make,say,way,really,did,time,good
Topic 2: appreciated,information,email,help,info,looking,hi,advance,mail,thanks
Topic 3: lord,church,christians,christian,believe,faith,christ,bible,jesus,god
Topic 4: algorithm,public,escrow,use,government,keys,clipper,encryption,chip,key
Topic 5: mac,cd,floppy,controller,ide,hard,drives,disk,scsi,drive
Topic 6: 50,20,price,condition,offer,shipping,10,new,sale,00
Topic 7: problem,running,using,use,program,window,files,dos,file,windows
Topic 8: teams,win,hockey,play,season,players,year,games,team,game
Topic 9: article,ftp,cc,university,cs,soon,banks,gordon,pitt,edu
Topic 10: oil,new,speed,good,miles,dealer,engine,bike,cars,car
Topic 11: ram,color,bus,driver,vga,cards,drivers,monitor,video,card
Topic 12: ftp,doesn,help,new,program,work,mean,anybody,know,does
Topic 13: months,yeah,couple,just,right,tried,heard,seen,got,ve
Topic 14: gay,militia,gun,guns,right,rights,government,don,think,people
Topic 15: group,listen,wanted,though